In [ ]:
from enceladus.jobs import TestingPipeline

ppg, vpg, apg, abp = TestingPipeline(
    data_path='/home/cam/Documents/database_tools/data/data-2022-11-08/mimic3/',
    scaler_path='/home/cam/Documents/database_tools/data/data-2022-11-08/mimic3/scalers_MinMax.pkl',
    scaler_type='MinMax',
    model_path='heartfelt/Enceladus/model-vital-bee-206:v19',
).run()

In [ ]:
import numpy as np
from google.cloud import aiplatform

GOOGLE_CLOUD_PROJECT = 'heartfelt-0'
GOOGLE_CLOUD_REGION = 'us-central1'
ENDPOINT_ID = 4207052545266286592

# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

# Set data values for the prediction request.
instances = [{'ppg': ppg[0, :].reshape(256, 1).tolist(),
              'vpg': vpg[0, :].reshape(256, 1).tolist(),
              'apg': apg[0, :].reshape(256, 1).tolist(),
}]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
# Send a prediction request and get response.
response = client.predict(endpoint=endpoint, instances=instances, parameters={'signature_name': 'serving_default'})

pred = np.array(response.predictions[0]).flatten()

In [ ]:
import pandas as pd

pd.options.plotting.backend = 'plotly'

pd.Series(pred).plot()

In [ ]:
ppg[0, :].reshape(1, 256).tolist()

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl

with open('model-vital-bee-206:v19_error.pkl', 'rb') as f:
    data = pkl.load(f)

print(np.mean(data['sbp_abs_err']), np.mean(data['dbp_abs_err']))

In [ ]:
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'

fig = go.Figure()
fig.add_trace(go.Histogram(x=data['sbp_err'], name='SBP Error'))
fig.add_trace(go.Histogram(x=data['dbp_err'], name='DBP Error'))

fig.update_layout(
        barmode='overlay',
        title={
            'text': 'Model Prediction Error Histogram',
            'font': {'size': 20},
        },
        xaxis={'title': 'Error (mmHg)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Number of Samples', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig
fig.update_traces(opacity=0.75)

fig

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from plotly.subplots import make_subplots

with open('model-vital-bee-206:v19_predictions.pkl', 'rb') as f:
    data = pkl.load(f)
    
ppg, vpg, apg, abp, pred = data.values()

In [ ]:
i = 18

In [ ]:
fig = make_subplots(rows=2, cols=1, start_cell='top-left')

# Add inputs
fig.add_scatter(
    y=ppg[i, :],
    name='PPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=vpg[i, :] * 10,
    name='VPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=apg[i, :] * 50,
    name='APG',
    row=1,
    col=1,
)

# Add ABP and prediction
fig.add_scatter(
    y=abp[i, :],
    name='ABP (Ground Truth)',
    row=2,
    col=1
)
fig.add_scatter(
    y=pred[i, :],
    name='Prediction',
    row=2,
    col=1,
)

fig.update_layout(
        title={
            'text': 'Model Inputs (PPG, VPG, APG) and Module Output w/ True ABP',
            'font': {'size': 20},
        },
        xaxis2={'title': 'Time (samples)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Amplitude (V)', 'titlefont':{'size': 15}},
        yaxis2={'title': 'Amplitude (mmHg)', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig